# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("cities.csv", encoding = "utf-8")
weather_data.head()

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC)
0,Tuktoyaktuk,69.4541,-133.0374,-34.60,67,75,11.50,CA,1612477285
1,Mataura,-46.1927,168.8643,60.24,56,36,3.87,NZ,1612477562
2,Cidreira,-30.1811,-50.2056,70.14,77,57,21.21,BR,1612477562
3,Saskylakh,71.9167,114.0833,-40.85,76,0,3.58,RU,1612477562
4,Atuona,-9.8000,-139.0333,78.82,69,59,10.92,PF,1612477563


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_data[["Latitude", "Longitude"]].astype(float)
humidity = weather_data["Humidity (%)"].astype(float)

In [5]:
fig = gmaps.figure(zoom_level=1.9, center=(25,0))

heat_layer = gmaps.heatmap_layer(locations,
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# reduce dataframe to include only cities matching a certain criteria
# max temp>=75, <85
# wind speed<5mph
# cloudiness<= 5

reduced_wd = weather_data.loc[(weather_data["Max Temp. (F)"]>=75) &
                             (weather_data["Max Temp. (F)"]<85) &
                             (weather_data["Wind Speed (mph)"]<5) &
                              (weather_data["Cloudiness (%)"]<=15)]
reduced_wd

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC)
127,Aboso,5.3617,-1.9473,76.32,92,1,3.65,GH,1612477592
318,Guánica,17.9716,-66.9080,80.01,76,9,1.99,PR,1612477636
478,Port Macquarie,-31.4333,152.9167,78.01,78,15,1.01,AU,1612477674


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = reduced_wd.copy()
hotel_df["Hotel Name"] = ""
params = {"radius": 5000,
         "types": "hotel",
          "key":g_key
         }
hotel_df.dropna()
hotel_df

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC),Hotel Name
127,Aboso,5.3617,-1.9473,76.32,92,1,3.65,GH,1612477592,
318,Guánica,17.9716,-66.9080,80.01,76,9,1.99,PR,1612477636,
478,Port Macquarie,-31.4333,152.9167,78.01,78,15,1.01,AU,1612477674,


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [15]:
# https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
#     print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except:
        print("Missing field/result... skipping.")
hotel_df

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC),Hotel Name
127,Aboso,5.3617,-1.9473,76.32,92,1,3.65,GH,1612477592,Prestea- Huni Valley District
318,Guánica,17.9716,-66.9080,80.01,76,9,1.99,PR,1612477636,Guanica
478,Port Macquarie,-31.4333,152.9167,78.01,78,15,1.01,AU,1612477674,Port Macquarie


In [16]:
markers = gmaps.marker_layer(locations, info_box_content = [f"{hotel}" for hotel in hotel_info])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))